In [1]:
#import necessary libraries

import numpy as np
import scipy as sp

In [ ]:
"""
Definining the Operators
- Morse Hamiltonian
- Always-on Hamiltonian
- Displacement Gate
- Qubit xy Rotation Gate


make time a constant parameter

how to define the 

optimizing time

"""



# Parameters for Morse Time Evolution
time = 2
mass = 1
diss_energy = 8
width_param = 0.8
equib_length = 0

hbar = 1    
# 6.62607015 * (10 ** (-34))

# Truncation for the oscillator Fock space
N = np.sqrt(2 * mass * diss_energy) / (width_param * hbar)
N = (int) (N - (1/2))

I_q = np.eye(2) #Identity operator for qubits
I_o = np.eye(N) #Identity operator for qumodes

a = np.diag(np.sqrt(np.arange(1, N)), 1)  # Annihilation operator
adag = a.T.conj()                         # Creation operator
n_op = adag @ a                           # Photon number operator

# Pauli Matrices for qubit
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

omega = 1.0  # Oscillator frequency
chi = 0.01   # Qubit-oscillator coupling

H_always_on = (chi * sigma_z + omega * np.eye(2))  # 2x2 matrix
H_always_on = np.kron(H_always_on, n_op)           # Full H'

# Always-on Time Evolution Gate
def H_On_Evo(t):
    return sp.linalg.expm(-1j * H_always_on * t)

#Dispacement gate 
def displacement(alpha):
    A = alpha * adag - np.conj(alpha) * a
    return sp.linalg.expm(A)

def D_full(alpha):
    return np.kron(I_q, displacement(alpha))

#Qubit XY Rotation Gate
def R_phi(theta, phi):
    op = (np.cos(phi) * sigma_x + np.sin(phi) * sigma_y)
    return sp.linalg.expm(-1j * theta / 2 * op)

def R_full(theta, phi):
    return np.kron(R_phi(theta, phi), I_o)


#Morse Hamiltonian
def H_Morse(u ,de, b, x0):
    """
    Morse Hamiltonian Gate

    Args:
        u (real): reduced mass of diatomic system
        de (real): dissociation energy
        b (real): width parameter
        x0 (real): equilibrium bond length

    Returns:
        csc_matrix: operator matrix
    """
    m_omega = np.sqrt(2 * de * (b ** 2) / u)
    X_op = (a + adag) / np.sqrt(2)
    P_op = 1j * (a - adag) / (np.sqrt(2))
    x_op = X_op * np.sqrt(hbar / (u * m_omega))
    p_op = P_op * np.sqrt(hbar * u * omega)

    kin_op = (p_op @ p_op) / (2 * u)

    exp_term = sp.linalg.expm(-1 * b * (x_op - x0 * np.eye(N)))
    mp_op = de * ((np.eye(N) - exp_term) @ (np.eye(N) - exp_term))

    full_m = kin_op + mp_op

    full_m = np.kron(I_q, full_m) #Embed into full qubit-oscillator hilbert space

    return full_m

def MH_Evo(t, u, de, b, x0):
    return sp.linalg.expm(-1j * H_Morse(u, de, b, x0) * t)

In [8]:
# Build the Gate Sequence

def gate_seq(params, d):
    """
    Builds the Gate Sequence

    Params: Each layer is a list of 5 parameters
    1. alpha_real - real part of alpha which shifts position
    2. alpha_imag - imaginary part of alpha which shifts momentum
    3. theta - one of the parameters for the xy qubit rotation gate
    4. phi - one of the parameters for the xy qubit rotaation gate
    5. t - time parameter for always-on hamiltonian
    
    """
    U = np.eye(2 * N, dtype=complex)
    for j in range(d):
        alpha_real = params[4*j]
        alpha_imag = params[4*j+1]
        theta = params[4*j+2]
        phi = params[4*j+3]
        t = params[4*j+4]
        
        D = D_full(alpha_real + 1j * alpha_imag)
        R = R_full(theta, phi)
        V = H_On_Evo(t)

        U = U @ V @ R @ D
    return U

### 🔢 Step-by-Step Breakdown of Fidelity Calculation

#### 1. `U.conj().T @ U_target`
This computes the Hermitian adjoint (conjugate transpose) of `U` multiplied by `U_target`:

$$
U^\dagger U_{\text{target}}
$$

This checks how well your the unitary matches the target.

---

#### 2. `np.trace(...)`
This computes the trace of the product:

$$
\mathrm{Tr}(U^\dagger U_{\text{target}})
$$

If the unitaries are equal, this trace equals the Hilbert space dimension.

---

#### 3. `np.abs(...) ** 2`
This gives the squared modulus of the trace:

$$
\left| \mathrm{Tr}(U^\dagger U_{\text{target}}) \right|^2
$$

This is a real, non-negative value that reflects overall overlap.

---

#### 4. `/ (dim ** 2)`
This normalizes the result by the square of the Hilbert space dimension:

$$
\text{fidelity} = \frac{ \left| \mathrm{Tr}(U^\dagger U_{\text{target}}) \right|^2 }{ \text{dim}^2 }
$$

This ensures the fidelity lies between 0 and 1, where:
- `1` means perfect overlap (unitaries are identical),
- `0` means no overlap.


In [9]:
# Define the Cost Function
def fidelity_loss(params, d, U_target):
    U = gate_seq(params, d)
    dim = U.shape[0]
    fid = np.abs(np.trace(U.conj().T @ U_target))/ (dim)
    return 1 - fid


In [ ]:
# Optimize

d = 10                                           # number of gate layers
num_params = d * 5                              # [Re(α), Im(α), θ, φ, t] per gate

# Provides small initial guess near 0
init_guess = np.random.rand(num_params) * 0.1   

# Create an instance of the morse time evolution
morse_to_optimize = MH_Evo(t=time, u=mass, de=diss_energy, b=width_param, x0=equib_length)

result = sp.optimize.minimize(
    #optimize will make guesses for parameteres of fidelity_loss that minimize its return values
    fidelity_loss,          
    init_guess,
    args=(d, morse_to_optimize),
    method='BFGS',
    options={'disp': True}
)

optimal_params = result.x

# Print the Optimized Parameters

omega = 1.0  # Oscillator frequency
chi = 0.1    # Qubit-oscillator coupling

def print_optimal_params(params, d):
    print(f"Fock Space Truncation         = {N}")
    print()
    print("Oscillator Paremeters:")
    print(f"    oscillator frequency      = {omega}")
    print(f"    qubit-oscillator coupling = {chi}")
    print()
    print("Morse Parameters Used:")
    print(f"    hbar           = {hbar}")
    print(f"    time           = {time}")
    print(f"    mass           = {mass}")
    print(f"    diss_energy    = {diss_energy}")
    print(f"    width_param    = {width_param}")
    print(f"    equilib_length = {equib_length}")

    print()
    print("-" * 30)

    print("Optimized Paramters")
    for i in range(d):
        re_alpha = params[i * 5 + 0]
        im_alpha = params[i * 5 + 1]
        theta    = params[i * 5 + 2]
        phi      = params[i * 5 + 3]
        t        = params[i * 5 + 4]
        
        print(f"Gate {i+1}:")
        print(f"  α     = {re_alpha:.4f} + {im_alpha:.4f}j")
        print(f"  θ     = {theta:.4f}")
        print(f"  φ     = {phi:.4f}")
        print(f"  t     = {t:.4f}")
        print("-" * 30)

# After optimization
print_optimal_params(optimal_params, d)


Optimization terminated successfully.
         Current function value: 0.000021
         Iterations: 1812
         Function evaluations: 99807
         Gradient evaluations: 1957
Fixed Paramters
Fock Space Truncation     = 4
Oscillator Paremeters Used:
    oscillator frequency      = 1.0
    qubit-oscillator coupling = 0.1
Morse Parameters Used:
    hbar           = 1
    time           = 2
    mass           = 1
    diss_energy    = 8
    width_param    = 0.8
    equilib_length = 0
------------------------------
Optimized Paramters
Gate 1:
  α     = -1.2842 + 0.9149j
  θ     = -0.2475
  φ     = -0.0803
  t     = 0.3561
------------------------------
Gate 2:
  α     = -2.1675 + 1.2647j
  θ     = 0.0714
  φ     = 0.2984
  t     = 1.0830
------------------------------
Gate 3:
  α     = -3.2744 + -0.0517j
  θ     = 0.3215
  φ     = -1.4185
  t     = 0.0243
------------------------------
Gate 4:
  α     = 0.8388 + -0.5707j
  θ     = 0.4896
  φ     = 0.4242
  t     = -0.0642
---------------